# Acquire the discography data of various artists from the Spotify Web API and obtain their corresponding touring/upcoming events data from the Ticketmasters Discovery API

- Create a developer account with Spotify (https://developer.spotify.com/) and make a dashboard application. From this  application obtain the Client ID and Client Secret required to make requests
- Use Spotipy, a lightweight Python library for the Spotify Web API to acquire discography data (https://spotipy.readthedocs.io/en/master/)
- Create an account with the Ticketmaster (https://developer.ticketmaster.com/products-and-docs/apis/discovery-api/v2/) to use their Discovery API. There is a limit of 5000 API calls per day and rate limitation of 5 requests per second
- Format and transform the acquired artist, album, track and event dataframes into a relational database for SQL analysis

In [1]:
#import libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import urllib.request, urllib.parse, urllib.error
import ssl
import pandas as pd
import numpy as np
import json
from collections import defaultdict
import time
import sqlite3

In [2]:
def artist_album_track_tables(names):
    
    artist_data = defaultdict(list) 
    album_data = defaultdict(list)   
    track_data = defaultdict(list)  
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="##########################",
                                                           client_secret="##########################"))
    #primary id for SQL analysis
    artist_pid = 0
    album_pid = 0
    track_pid = 0
    
    #lets first obtain artist data:
    for name in names:
        artist_pid += 1
        artist_data['artist_pid'].append(artist_pid)
        
        
        name = "artist:" + name
        artist_id = sp.search(q=name, limit=1,offset = 0)['tracks']['items'][0]['album']['artists'][0]['id']
        artist_query = sp.artist(artist_id)
        
        artist_data['followers'].append(artist_query['followers']['total'])
        artist_data['genre'].append(artist_query['genres'][0])
        artist_data['artist_name'].append(artist_query['name'])
        artist_data['popularity'].append(artist_query['popularity'])

        albums_query = sp.artist_albums(artist_id,album_type = "album",limit = 50)
        artist_data['total_albums'].append(albums_query['total'])
        
        #lets extract album data:
        for i in range(albums_query['total']):
            album_pid += 1
            album_data['album_pid'].append(album_pid)
            album_data['artist_id'].append(artist_pid)
            
            album_id = albums_query['items'][i]["id"]
            query_album = sp.album(album_id)
            
            album_data['album_name'].append(query_album['name'])
            album_data['total_tracks'].append(query_album['total_tracks'])
            album_data['label'].append(query_album['label'])
            album_data['release_date'].append(query_album['release_date'])
            album_data['popularity'].append(query_album['popularity'])
        
            #lets now extract track data:
            for j in range(len(query_album['tracks']['items'])):
                track_pid += 1
                track_data['track_pid'].append(track_pid)
                track_data['album_id'].append(album_pid)
                track_data['artist_id'].append(artist_pid)
                
                track_id = query_album['tracks']['items'][j]['id']
                query_track = sp.track(track_id)
                track_data['track_name'].append(query_track['name'])
                track_data['popularity'].append(query_track['popularity'])
                track_data['duration_ms'].append(query_track['duration_ms'])
                track_data['track_number'].append(query_track['track_number'])
                track_data['disc_number'].append(query_track['disc_number'])
                track_data['explicit'].append(query_track['explicit'])

                query_audiofeatures = sp.audio_features(track_id)
                track_data['danceability'].append(query_audiofeatures[0]['danceability'])
                track_data['energy'].append(query_audiofeatures[0]['energy'])
                track_data['key'].append(query_audiofeatures[0]['key'])
                track_data['loudness'].append(query_audiofeatures[0]['loudness'])
                track_data['mode'].append(query_audiofeatures[0]['mode'])
                track_data['speechiness'].append(query_audiofeatures[0]['speechiness'])
                track_data['acousticness'].append(query_audiofeatures[0]['acousticness'])
                track_data['instrumentalness'].append(query_audiofeatures[0]['instrumentalness'])
                track_data['liveness'].append(query_audiofeatures[0]['liveness'])
                track_data['valence'].append(query_audiofeatures[0]['valence'])
                track_data['tempo'].append(query_audiofeatures[0]['tempo'])
                track_data['time_signature'].append(query_audiofeatures[0]['time_signature'])
          
        print('{}, albums:{}'.format(name,albums_query['total']))
    return([artist_data,album_data,track_data])

In [3]:
def ticketmaster_events(artists,genres):
    
    api = "##########################"
    event_data = defaultdict(list) 
    artist_pid = 0
    event_pid = 0
    for i in range(len(artists)):
        artist_pid += 1
        musician = urllib.parse.quote(artists[i])
        genre = urllib.parse.quote(genres[i])
        
        serviceurl = f'https://app.ticketmaster.com/discovery/v2/events.json?keyword={musician}&size={99}'\
                     f'&classificationName={genre}&apikey={api}'
        uh = urllib.request.urlopen(serviceurl)
        data = uh.read().decode()
        js_pg1 = json.loads(data) 
        time.sleep(1)
        
        for i in range(js_pg1['page']['totalElements']):
            if js_pg1['page']['totalElements'] > 0:
                
                try:
                    event_data['venue_address'].append(js_pg1['_embedded']['events'][i]['_embedded']['venues'][0]['address']['line1'])
                except:
                    continue   
                try:
                    event_data['venue_name'].append(js_pg1['_embedded']['events'][i]['_embedded']['venues'][0]['name'])
                except:
                    event_data['venue_name'].append(None)    
                try:
                    event_data['venue_location_latitude'].append(js_pg1['_embedded']['events'][i]['_embedded']['venues'][0]['location']['latitude'])
                    event_data['venue_location_longitude'].append(js_pg1['_embedded']['events'][i]['_embedded']['venues'][0]['location']['longitude'])
                except:
                    event_data['venue_location_latitude'].append(None)
                    event_data['venue_location_longitude'].append(None)
                try:
                    event_data['event_DateTime'].append(js_pg1['_embedded']['events'][i]['dates']['start']['dateTime'])
                except:
                    event_data['event_DateTime'].append(None)
                
                event_pid += 1
                event_data['event_pid'].append(event_pid)
                event_data['artist_id'].append(artist_pid)
                event_data['event_name'].append(js_pg1['_embedded']['events'][i]['name'])
                event_data['venue_country'].append(js_pg1['_embedded']['events'][i]['_embedded']['venues'][0]['country']['name'])
                event_data['venue_country_code'].append(js_pg1['_embedded']['events'][i]['_embedded']['venues'][0]['country']['countryCode'])
                event_data['venue_city'].append(js_pg1['_embedded']['events'][i]['_embedded']['venues'][0]['city']['name'])
                event_data['venue_postalCode'].append(js_pg1['_embedded']['events'][i]['_embedded']['venues'][0]['postalCode'])
        print('artist:{}, events:{}'.format(artists[artist_pid-1], js_pg1['page']['totalElements']))
    return(event_data)

In [4]:
artists = ["Post Malone","The Weeknd","Kid Cudi", "NIKI","Kanye West","Dua Lipa","Arkells","Arctic Monkeys",
           "Ed Sheeran", "H.E.R."]
genres = ['hip hop','hip hop','hip hop','r&b','rap','pop','indie','rock','pop','r&b']

In [5]:
ten_artists = artist_album_track_tables(artists)

artist:Post Malone, albums:13
artist:The Weeknd, albums:32
artist:Kid Cudi, albums:29
artist:NIKI, albums:7
artist:Kanye West, albums:30
artist:Dua Lipa, albums:16
artist:Arkells, albums:21
artist:Arctic Monkeys, albums:14
artist:Ed Sheeran, albums:32
artist:H.E.R., albums:6


In [6]:
events = ticketmaster_events(artists,genres)

artist:Post Malone, events:51
artist:The Weeknd, events:9
artist:Kid Cudi, events:9
artist:NIKI, events:11
artist:Kanye West, events:0
artist:Dua Lipa, events:7
artist:Arkells, events:0
artist:Arctic Monkeys, events:14
artist:Ed Sheeran, events:4
artist:H.E.R., events:2


In [7]:
df_artists = pd.DataFrame(dict(ten_artists[0]))
df_albums = pd.DataFrame(dict(ten_artists[1]))
df_tracks = pd.DataFrame(dict(ten_artists[2]))
df_events = pd.DataFrame(dict(events))

In [8]:
df_artists = df_artists[["artist_pid","artist_name","genre","total_albums","followers","popularity"]]
df_artists= df_artists.rename(columns = {'artist_pid':'id'})
df_artists

,id,artist_name,genre,total_albums,followers,popularity
0,1,Post Malone,dfw rap,13,38035670,89
1,2,The Weeknd,canadian contemporary r&b,32,49168694,93
2,3,Kid Cudi,hip hop,29,5992511,81
3,4,NIKI,indonesian r&b,7,1858406,76
4,5,Kanye West,chicago rap,30,18133628,90
5,6,Dua Lipa,dance pop,16,36080896,88
6,7,Arkells,canadian indie,21,164243,54
7,8,Arctic Monkeys,garage rock,14,15800777,86
8,9,Ed Sheeran,pop,32,101951441,90
9,10,H.E.R.,pop,6,5508278,75


In [9]:
df_albums = df_albums.rename(columns = {'album_pid':'id'})
df_albums.at[147,'release_date'] = "2008-01-01"
df_albums

,id,artist_id,album_name,total_tracks,label,release_date,popularity
0,1,1,Twelve Carat Toothache (Deluxe),16,Mercury Records/Republic Records,2022-06-07,71
1,2,1,Twelve Carat Toothache (Deluxe),16,Mercury Records/Republic Records,2022-06-07,48
2,3,1,Twelve Carat Toothache,14,Mercury Records/Republic Records,2022-06-03,82
3,4,1,Twelve Carat Toothache,14,Mercury Records/Republic Records,2022-06-03,58
4,5,1,Hollywood's Bleeding,17,Republic Records,2019-09-06,81
...,...,...,...,...,...,...,...
195,196,10,Back of My Mind,21,MBK Entertainment/RCA Records,2021-06-18,39
196,197,10,I Used To Know Her,19,MBK Entertainment/RCA Records,2019-08-30,64
197,198,10,H.E.R.,21,MBK Entertainment/RCA Records,2017-10-20,64
198,199,10,H.E.R. Volume 2,8,MBK Entertainment/RCA Records,2017-06-16,47


In [10]:
df_tracks = df_tracks.rename(columns = {'track_pid' : 'id'})
df_tracks

,id,album_id,artist_id,track_name,popularity,duration_ms,track_number,disc_number,explicit,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,1,1,1,Reputation,65,248177,1,1,True,0.314,...,2,-5.726,1,0.0311,0.67100,0.000000,0.1810,0.364,180.119,4
1,2,1,1,Cooped Up (with Roddy Ricch),65,185857,2,1,True,0.669,...,0,-5.619,1,0.1530,0.03800,0.000000,0.1130,0.377,93.923,4
2,3,1,1,Lemon Tree,66,243486,3,1,True,0.611,...,11,-6.897,0,0.0301,0.08300,0.000000,0.4320,0.181,116.106,4
3,4,1,1,Wrapped Around Your Finger,65,193565,4,1,False,0.744,...,2,-5.435,1,0.0287,0.04930,0.000000,0.3170,0.539,119.966,4
4,5,1,1,I Like You (A Happier Song) (with Doja Cat),67,192840,5,1,True,0.733,...,5,-6.009,1,0.0751,0.12100,0.000000,0.1210,0.472,100.964,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3149,3150,200,10,Facts,30,218723,3,1,False,0.739,...,10,-14.191,0,0.1400,0.58300,0.009990,0.0936,0.383,126.996,4
3150,3151,200,10,U,37,178935,4,1,False,0.647,...,4,-7.953,0,0.0502,0.00767,0.000060,0.1170,0.271,109.942,4
3151,3152,200,10,Focus,57,200194,5,1,False,0.597,...,1,-13.483,0,0.0576,0.25500,0.000019,0.1080,0.090,87.082,4
3152,3153,200,10,Jungle,45,304653,6,1,False,0.653,...,4,-12.324,0,0.0709,0.70200,0.000215,0.0963,0.367,99.732,4


In [11]:
df_events = df_events[["event_pid","artist_id","event_name","venue_name","venue_country","venue_country_code",
                       "venue_city","venue_postalCode","venue_address","venue_location_latitude",
                       "venue_location_longitude","event_DateTime"]]
df_events = df_events.rename(columns = {'event_pid' : 'id'})
df_events

,id,artist_id,event_name,venue_name,venue_country,venue_country_code,venue_city,venue_postalCode,venue_address,venue_location_latitude,venue_location_longitude,event_DateTime
0,1,1,Post Malone: Twelve Carat Tour,T-Mobile Arena,United States Of America,US,Las Vegas,89109,3780 Las Vegas Blvd.,36.124401,-115.145103,2022-11-12T04:00:00Z
1,2,1,Post Malone: Twelve Carat Tour,Toyota Center - TX,United States Of America,US,Houston,77002,1510 Polk St,29.759399,-95.362999,2022-10-26T01:00:00Z
2,3,1,Post Malone: Twelve Carat Tour,Wells Fargo Center-PA,United States Of America,US,Philadelphia,19148,3601 South Broad St.,39.910999,-75.153397,2022-10-07T00:00:00Z
3,4,1,Post Malone: Twelve Carat Tour,Nationwide Arena,United States Of America,US,Columbus,43215,200 West Nationwide Blvd.,39.96939661,-83.00636203,2022-09-19T00:00:00Z
4,5,1,Post Malone: Twelve Carat Tour,Rocket Mortgage FieldHouse,United States Of America,US,Cleveland,44115,One Center Court,41.492401,-81.671204,2022-09-28T00:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...
100,101,9,Ed Sheeran + - = ÷ x Tour,Eden Park,New Zealand,NZ,Auckland,1024,Reimers Ave,-36.8761749,174.7436981,None
101,102,9,Ed Sheeran + - = ÷ x Tour,Optus Stadium,Australia,AU,Burswood,6100,333 Victoria Park Drive,-31.951204,115.889132,None
102,103,9,2022 Formula 1 Aramco United States Grand Prix...,Circuit of The Americas,United States Of America,US,Austin,78617,9201 Circuit of The Americas Blvd,30.13526827,-97.63566001,2022-10-22T13:00:00Z
103,104,10,THE INCITERS w/ Lou Lou and Her Spizy Boyz plu...,Moe's Alley,United States Of America,US,Santa Cruz,95065,1535 Commercial Way,36.9875105,-121.9828327,2022-10-29T04:00:00Z


In [12]:
df_artists.to_csv('/home/mslobody/Desktop/Spotify_Ticketmaster/artists.tsv', sep = '\t')  
df_albums.to_csv('/home/mslobody/Desktop/Spotify_Ticketmaster/albums.tsv', sep = '\t')  
df_tracks.to_csv('/home/mslobody/Desktop/Spotify_Ticketmaster/tracks.tsv', sep = '\t')  
df_events.to_csv('/home/mslobody/Desktop/Spotify_Ticketmaster/events.tsv', sep = '\t')  

In [13]:
#Create a Relational Database in sqlite3
conn = sqlite3.connect('artists_albums_tracks_events.sqlite') 
cur = conn.cursor()

cur.executescript('''  
DROP TABLE IF EXISTS Artists;
DROP TABLE IF EXISTS Albums;
DROP TABLE IF EXISTS Tracks;
DROP TABLE IF EXISTS Events;

CREATE TABLE Artists (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_name TEXT UNIQUE,
    genre TEXT,
    total_albums INTEGER,
    followers INTEGER,
    popularity INTEGER
);
CREATE TABLE Albums (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id INTEGER,
    album_name TEXT,
    total_tracks INTEGER,
    label TEXT,
    release_date DATETIME,
    popularity INTEGER
);
CREATE TABLE Tracks (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    album_id INTEGER,
    artist_id INTEGER,
    track_name TEXT,
    popularity INTEGER,
    duration_ms INTEGER,
    track_number INTEGER,
    disc_number INTEGER,
    explicit BOOL,
    danceability FLOAT,
    energy FLOAT,
    key INTEGER,
    loudness FLOAT,
    mode INTEGER,
    speechiness FLOAT,
    acousticness FLOAT,
    instrumentalness FLOAT,
    liveness FLOAT,
    valence FLOAT,
    tempo FLOAT,
    time_signature INTEGER
);
CREATE TABLE Events (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id INTEGER,
    event_name TEXT,
    venue_name TEXT,
    venue_country TEXT,
    venue_country_code TEXT,
    venue_city TEXT,
    venue_postalCode TEXT,
    venue_address TEXT,
    venue_location_latitude FLOAT,
    venue_location_longitude FLOAT,
    event_DateTime DATETIME
);
''')

df_artists.to_sql('Artists',conn, if_exists = "replace", index = False)
df_albums.to_sql('Albums', conn, if_exists = "replace", index = False)
df_tracks.to_sql('Tracks', conn, if_exists = "replace", index = False)
df_events.to_sql('Events', conn, if_exists = "replace", index = False)

### Install the DB Browser for SQLite Version 3.11.2 to run "2_SQL_Exploratory_Analysis.sql"